In [ ]:
!rm -rf /usr/local/lib/python3.11/dist-packages/~vidia*


In [ ]:
# Downgrade numpy if you want to strictly match
!pip install numpy==1.24.4

# Then, restart runtime (Menu > Runtime > Restart Runtime) #

In [ ]:
!pip install --upgrade gensim
!pip install git+https://github.com/facebookresearch/fastText.git

  Cloning https://github.com/facebookresearch/fastText.git to /tmp/pip-req-build-ecuc4yj4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fastText.git /tmp/pip-req-build-ecuc4yj4
  Resolved https://github.com/facebookresearch/fastText.git to commit 1142dc4c4ecbc19cc16eee5cdd28472e689267e6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from google.colab import files
files.upload()


KeyboardInterrupt: 

In [ ]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)
os.rename('kaggle.json', '/root/.kaggle/kaggle.json')
os.chmod('/root/.kaggle/kaggle.json', 600)


In [ ]:
!kaggle datasets download -d manidaaw/tamil-oscar-corpus


Dataset URL: https://www.kaggle.com/datasets/manidaaw/tamil-oscar-corpus
License(s): unknown


In [ ]:
import zipfile
with zipfile.ZipFile("tamil-oscar-corpus.zip", 'r') as zip_ref:
    zip_ref.extractall("tamil_oscar")

In [ ]:
def read_corpus(path):
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                yield line.split()
corpus_path = "/content/tamil_oscar/ta_part_1.txt.gz"
sentences = list(read_corpus(corpus_path))
print("Number of sentences loaded:", len(sentences))
print("Sample:", sentences[0])

Number of sentences loaded: 2470684
Sample: ['ஒலிம்பிக்', 'போட்டிகள்', 'நடந்த', 'இடங்கள்', '1.', '1896', '-', 'ஏதென்ஸ்,', 'கிரீஸ்', '2.', '1900', '-', 'பாரிஸ்,', 'பிரான்ஸ்', '3.', '1904', '-', 'செயின்', 'லூயிஸ்,', 'அமெரிக்கா', '4.', '1908', '-', 'லண்டன்,பிரிட்டன்', '5.', '1912', '-', 'ஸ்டோக்ஹோம்,', 'சுவீடன்', '6.', '1920', '-', 'ஆண்ட்வெர்ப்,', 'பெல்ஜியம்', '7.', '1924', '-', 'பாரிஸ்,', 'பிரான்ஸ்', '8.', '1928', '-', 'ஆம்ஸ்டர்டாம்,', 'ஹாலந்து', '9.', '1932', '-', 'லாஸ்,', 'ஏஞ்சல்ஸ்', '10.', '1936', '-', 'பெர்லின்,', 'ஜெர்மனி', '11.', '1948', '-', 'லண்டன்,', 'இங்கிலாந்து', '12.', '1952', '-', 'ஹல்சின்கி,', 'பின்லாந்து', '13.', '1956', '-', 'மேபோர்ன்,ஆஸ்திரேலியா', '14.', '1960', '-', 'ரோம்,', 'இத்தாலி', '15.', '1964', '-', 'டோக்கியோ,', 'ஜப்பான்', '16.', '1968', '-', 'மெக்சிகோ,', 'மெக்ஸிக்கோ', '17.', '1972', '-', 'மியூனிக்,', 'ஜெர்மனி', '18.', '1976', '-', 'மான்ட்ரியல்,', 'கனடா', '19.', '1980', '-', 'மாஸ்கோ,', 'USSR', '20.', '1984', '-', 'லாஸ்', 'ஏஞ்சல்ஸ்,', 'அமெரிக்கா', '21.', '1988', '-'

In [ ]:
from gensim.models import Word2Vec

# Use a smaller subset for quicker training (adjust as needed)
short_sentences = sentences[:50000]  # Only use first 10,000 lines

# Train with faster settings
w2v_model = Word2Vec(
    sentences=short_sentences,
    vector_size=50,     # Smaller embedding size
    window=5,           # Context window
    min_count=10,       # Ignore rare words
    workers=4,          # Parallel training
    sg=0,               # Use CBOW (faster than skip-gram)
    epochs=5            # Fewer training iterations
)

# Save the model
w2v_model.save("tamil_word2vec_quick.model")

print("✅ Quick Word2Vec model trained and saved!")


✅ Quick Word2Vec model trained and saved!


In [ ]:
# Load the model (optional if you're continuing in the same session)
from gensim.models import Word2Vec
model = Word2Vec.load("tamil_word2vec_quick.model")

# Get vector for a word
print("📍 Vector for தமிழ்:", model.wv["தமிழ்"][:10])  # First 10 dimensions

# Find most similar words
similar_words = model.wv.most_similar("தமிழ்", topn=5)
print("🔍 Similar words to 'தமிழ்':")
for word, score in similar_words:
    print(f"{word} - {score:.4f}")


📍 Vector for தமிழ்: [ 0.10792143 -0.24503167 -1.2529924  -0.37729755  1.2537847   2.1221123
  0.5669657  -0.2654165  -0.09773123  2.2463973 ]
🔍 Similar words to 'தமிழ்':
இலக்கிய - 0.7994
மொழி - 0.7944
சினிமாவில் - 0.7787
சினிமா - 0.7655
சினிமாவின் - 0.7070


In [ ]:
# 💬 Input a Tamil sentence from the user
input_sentence = input("✍️ Enter a Tamil sentence: ")

# 🧩 Tokenize the sentence (basic whitespace-based split)
words = input_sentence.strip().split()

print("\n🔍 Word analysis:")
for word in words:
    if word in model.wv:
        print(f"\n📌 Word: {word}")
        print("🔹 Vector (first 5 dims):", model.wv[word][:5])

        similar_words = model.wv.most_similar(word, topn=3)
        print("🔗 Top 3 similar words:")
        for similar_word, score in similar_words:
            print(f"   ➤ {similar_word} ({score:.4f})")
    else:
        print(f"\n⚠️ Word '{word}' not found in vocabulary.")


✍️ Enter a Tamil sentence: தமிழ் ஒரு அழகான மொழி.

🔍 Word analysis:

📌 Word: தமிழ்
🔹 Vector (first 5 dims): [ 0.10792143 -0.24503167 -1.2529924  -0.37729755  1.2537847 ]
🔗 Top 3 similar words:
   ➤ இலக்கிய (0.7994)
   ➤ மொழி (0.7944)
   ➤ சினிமாவில் (0.7787)

📌 Word: ஒரு
🔹 Vector (first 5 dims): [-0.2968386   1.3143142  -0.46690714  1.3146205  -1.2328315 ]
🔗 Top 3 similar words:
   ➤ ஒரே (0.7826)
   ➤ இன்னொரு (0.7039)
   ➤ அதே (0.7025)

📌 Word: அழகான
🔹 Vector (first 5 dims): [-0.15006211  0.7148199  -0.62346435  0.19145738 -0.64105445]
🔗 Top 3 similar words:
   ➤ பக்கம் (0.9705)
   ➤ குரலில் (0.9636)
   ➤ பிடித்த (0.9603)

📌 Word: மொழி.
🔹 Vector (first 5 dims): [ 0.06857381  0.13303326 -0.19101429 -0.0338296  -0.00205726]
🔗 Top 3 similar words:
   ➤ மொழி, (0.9735)
   ➤ படைப்புகள் (0.9713)
   ➤ மொழியின் (0.9672)


In [ ]:
!pip install streamlit matplotlib
!apt-get -y install fonts-noto


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-noto-cjk fonts-noto-cjk-extra fonts-noto-color-emoji fonts-noto-core
  fonts-noto-extra fonts-noto-mono fonts-noto-ui-core fonts-noto-ui-extra
  fonts-noto-unhinted
The following NEW packages will be installed:
  fonts-noto fonts-noto-cjk fonts-noto-cjk-extra fonts-noto-color-emoji
  fonts-noto-core fonts-noto-extra fonts-noto-mono fonts-noto-ui-core
  fonts-noto-ui-extra fonts-noto-unhinted
0 upgraded, 10 newly installed, 0 to remove and 34 not upgraded.
Need to get 317 MB of archives.
After this operation, 790 MB of additi

In [ ]:
# Install Tamil font
!apt-get -y install fonts-noto
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Noto Sans Tamil'
plt.rcParams['axes.unicode_minus'] = False


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto is already the newest version (20201225-1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
import matplotlib.pyplot as plt

def plot_similar_words(word):
    if word not in model.wv:
        return f"⚠️ '{word}' not found in vocabulary.", None

    similar = model.wv.most_similar(word, topn=5)
    labels, scores = zip(*similar)

    # Plot
    plt.figure(figsize=(6, 4))
    plt.barh(labels, scores)
    plt.xlabel("Similarity Score")
    plt.title(f"Top 5 Similar Words to '{word}'")
    plt.gca().invert_yaxis()

    return f"🔍 Similar words to '{word}'", plt


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Use Noto Sans Tamil font for Tamil rendering
plt.rcParams['font.family'] = 'Noto Sans Tamil'


In [ ]:
!apt-get -y install fonts-noto fonts-noto-unhinted fonts-noto-color-emoji


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto is already the newest version (20201225-1build1).
fonts-noto-unhinted is already the newest version (20201225-1build1).
fonts-noto-unhinted set to manually installed.
fonts-noto-color-emoji is already the newest version (2.047-0ubuntu0.22.04.1).
fonts-noto-color-emoji set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
!sudo apt-get update
!sudo apt-get -y install fonts-noto fonts-noto-unhinted fonts-noto-color-emoji


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,723 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Pac

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Set Tamil font from system path (Linux path)
tamil_font = fm.FontProperties(fname="/usr/share/fonts/truetype/noto/NotoSansTamil-Regular.ttf")
plt.rcParams['font.family'] = tamil_font.get_name()


In [ ]:
!pip install streamlit numpy==1.24.4 gensim git+https://github.com/facebookresearch/fastText.git
!apt-get -y install fonts-noto


  Cloning https://github.com/facebookresearch/fastText.git to /tmp/pip-req-build-rhx3lknm
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fastText.git /tmp/pip-req-build-rhx3lknm
  Resolved https://github.com/facebookresearch/fastText.git to commit 1142dc4c4ecbc19cc16eee5cdd28472e689267e6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto is already the newest version (20201225-1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
%%writefile app.py
import streamlit as st
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# ✅ Tamil Font Handling (Linux + Windows fallback)
try:
    tamil_font = fm.FontProperties(fname="/usr/share/fonts/truetype/noto/NotoSansTamil-Regular.ttf")  # Linux path
except:
    tamil_font = fm.FontProperties(fname="C:/Windows/Fonts/NotoSansTamil-Regular.ttf")  # Windows path

plt.rcParams['font.family'] = tamil_font.get_name()
plt.rcParams['axes.unicode_minus'] = False

# ✅ Load Word2Vec model
model = Word2Vec.load("tamil_word2vec_quick.model")

# ✅ Sample intent dataset
intent_data = [
    ("வணக்கம், எப்படி இருக்கிறீர்கள்?", "வாழ்த்து"),
    ("டிக்கெட் பதிவு செய்யவேண்டும்", "டிக்கெட் பதிவு"),
    ("ஒரு படம் பார்க்கவேண்டும்", "பொழுது வினோதம்"),
    ("எனது டிக்கெட்டை ரத்து செய்யவும்", "டிக்கெட் ரத்து"),
    ("நன்றி உங்கள் உதவிக்கு", "நன்றி"),
    ("மாலை ஷோக்கு டிக்கெட் வேண்டும்", "டிக்கெட் பதிவு"),
    ("எனது டிக்கெட்டை முடக்கியுங்கள்", "டிக்கெட் ரத்து"),
    ("உங்கள் சேவைக்கு நன்றி", "நன்றி"),
    ("நான் படம் பார்க்க விரும்புகிறேன்", "பொழுது வினோதம்"),
    ("வணக்கம்", "வாழ்த்து")
]

# ✅ Vectorizer
def get_sentence_vector(sentence):
    words = sentence.strip().split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# ✅ Train classifier
X = [get_sentence_vector(text) for text, label in intent_data]
y = [label for _, label in intent_data]
le = LabelEncoder()
y_encoded = le.fit_transform(y)
clf = LogisticRegression(max_iter=1000)
clf.fit(X, y_encoded)

# ✅ Streamlit UI
st.title("Word embedding for tamil language(தமிழ் மொழிக்கான வார்த்தை உட்பொதித்தல்)")
st.markdown("தமிழ் உரையை உள்ளிடவும் – நாங்கள் நோக்கம் மற்றும் இணையான சொற்களை காட்டுவோம்:")

user_input = st.text_input("✍️ உங்கள் வாக்கியத்தை உள்ளிடவும்")

if user_input:
    vec = get_sentence_vector(user_input)
    pred = clf.predict([vec])[0]
    pred_label = le.inverse_transform([pred])[0]
    st.success(f"**கணிக்கப்பட்ட நோக்கம்:** {pred_label}")

    words = user_input.strip().split()
    for word in words:
        if word in model.wv:
            st.markdown(f"### 📌 சொல்: {word}")
            st.write("🔹 வெக்டார் (முதல் 5 அம்சங்கள்):", model.wv[word][:5])
            similar = model.wv.most_similar(word, topn=5)
            labels, scores = zip(*similar)

            # ✅ Tamil-compatible plot
            fig, ax = plt.subplots()
            ax.barh(labels, scores)
            ax.set_xlabel("ஒற்றுமை மதிப்பெண்", fontproperties=tamil_font)
            ax.set_title(f"🔗 '{word}' என்பதற்கு இணையான சொற்கள்", fontproperties=tamil_font)
            ax.invert_yaxis()
            for label in ax.get_yticklabels():
                label.set_fontproperties(tamil_font)
            st.pyplot(fig)
        else:
            st.warning(f"⚠️ '{word}' என்ற சொல் சொல்லகராதியில் இல்லை.")


Overwriting app.py


In [ ]:
!pip install pyngrok


In [ ]:
from pyngrok import conf, ngrok

# Paste your token inside the quotes
conf.get_default().auth_token = "2wawCEdTws3SUlBjgprxu4mTwLZ_jHzbLoc794PdtvaDaGYu"

# Kill any old tunnels
ngrok.kill()

# Run the Streamlit app
!streamlit run app.py &>/content/log.txt &

# Create tunnel
public_url = ngrok.connect(8501, "http")
print(f"🔗 Click here to open your app: {public_url}")


🔗 Click here to open your app: NgrokTunnel: "https://d39d-130-211-248-205.ngrok-free.app" -> "http://localhost:8501"
